In [1]:
import numpy as np 
import math
import cv2

desc1 = None
desc2 = None
def save_image(addr):
    #将视频的每一帧读取出来
    videoCapture = cv2.VideoCapture(addr)
    success, frame = videoCapture.read()
    pic_num = 0
    while success:
        frame=cv2.resize(frame,(540,960),0,0)
        pic_num = pic_num + 1
        address = 'D:/output/image' + str(pic_num)+ '.jpg'
        cv2.imwrite(address,frame)
        success, frame = videoCapture.read()
    print(pic_num)
    return pic_num


class camera:
    def __init__(self, width, height, fx, fy, cx, cy,
                 k1=0, k2=0, k3=0 ,p1=0,p2=0):
        #定义相机内参
        self.width = width
        self.height = height
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.distortion = [k1, k2, k3, p1, p2]
        
class system:
    def __init__(self, cam ):
        #初始化定位系统
        self.cam = cam
        self.current_frame = None
        self.last_frame = None
        self.src_pts = None
        self.dst_pts = None
        self.focal = cam.fx
        self.t = 0,0,0
        self.R = None
        self.pp = (cam.cx, cam.cy)
        self.detector = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
        self.px_ref = None
        
    def feature_detect(self):
        #特征点提取     
        kp2, st, err = cv2.calcOpticalFlowPyrLK(self.last_frame, self.current_frame, self.px_ref, None, **lk_params)
        st = st.reshape(st.shape[0])
        kp1 = self.px_ref[st == 1]
        kp2 = kp2[st == 1]
        self.src_pts = kp1
        self.dst_pts = kp2
        '''
        sift = cv2.xfeatures2d.SIFT_create()
        indexParams = dict(algorithm=0, trees=5)
        searchParams = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(indexParams,searchParams)
 #       if(pic_num == 2):
        kp1,desc1 = sift.detectAndCompute(self.last_frame,None)
        self.src_pts = np.float32([kp1[i].pt for i in range(1000)]).reshape(-1,2)
        
        kp2,desc2 = sift.detectAndCompute(self.last_frame,None)
        self.dst_pts = np.float32([kp2[i].pt for i in range(1000)]).reshape(-1,2)
       
      #  return src_pts, dst_pts

        raw_matches = matcher.knnMatch(desc1, desc2, 2)
        good = []
        for m,n in raw_matches:
            if m.distance < 0.7*n.distance:
                good.append(m)
        if len(good)>10:
            self.src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,2)
            self.dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,2)
        else:
            print("Not enough matches are found - %d/%d" % (len(good),10))
            matchesMask = None
     '''
        
    def find_location(self,pic_num):
        #单步计算旋转和平移量
     #   if pic_num == 2:
        self.px_ref = self.detector.detect(self.last_frame)
        self.px_ref = np.array([x.pt for x in self.px_ref], dtype=np.float32)
        self.feature_detect()
        E, mask = cv2.findEssentialMat(self.dst_pts, self.src_pts,focal=self.focal, pp=self.pp, method=cv2.RANSAC) 

        _, temp_R, temp_t, mask = cv2.recoverPose(E, self.dst_pts, self.src_pts, focal=self.focal, pp=self.pp)
        if pic_num == 2:
            self.t=temp_t
            self.R=temp_R
        else:
            self.t += self.R.dot(temp_t)
            self.R = temp_R.dot(self.R)
        
        
    def process(self,pic_num,camcam):
        #串起整体定位流程
        if pic_num == 1:
            return
        else:
            img1 = cv2.imread('D:/output/image' + str(pic_num-1)+ '.jpg')
            img2 = cv2.imread('D:/output/image' + str(pic_num)+ '.jpg')
            print(pic_num)
            self.last_frame = img1
            self.current_frame = img2
            self.find_location(pic_num)
            return
    
        
def is_R_correct(R):
    #判断求出来的旋转矩阵是否正确
    RT = np.transpose(R)
    RTR = np.dot(RT,R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - RTR)
    return n < 1e-6
'''        
def R_to_angle(R):
    #将旋转变量转化为欧拉角
    if not is_R_correct:
        print("R is incorrect.")
    else:
        sy = math.sqrt(R[0,0] * R[0,0] + R[1,0]*R[1,0])
        singular = sy < 1e-6
        if not singular:
            x = math.atan2(R[2,1], R[2,2])
            y = math.atan2(-R[2,0], sy)
            z = math.atan2(R[1,0],R[0,0])
            
        else:
            x = math.atan2(-R[1,2], R[1,1])
            y = math.atan2(-R[2,0], sy)
            z = 0
        return np.array([x,y,z])
'''    

lk_params = dict(winSize  = (21, 21), criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
#new_cam = camera(1920,1080,458, 457, 960,540)
#new_cam = camera(1080,1920,458, 457, 540,960)
camcam = cv2.VideoCapture(0)
cv2.namedWindow('MyCamera')
i=1
new_cam = camera(480,640,458,457,240,320)
#new_cam = camera(960,544,458,457,480,272)
new_system = system(new_cam)
location = np.zeros((1000,1000,3), dtype = np.uint8)
while True:
    success, frame = camcam.read()
    cv2.imshow('MyCamera',frame)
    address = 'D:/output/image' + str(i)+ '.jpg'
    cv2.imwrite(address,frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
    if i>1:

        #nn = save_image("25.mp4")
        f = open("out.txt", "w")
        new_system.process(i,camcam)
       # cv2.imshow('MyCamera',frame)
        cv2.waitKey(1)
        t = new_system.t
        R = new_system.R
    #    angle = R_to_angle(R)
        x,y,z = t[0]*3, t[1]*3, -t[2]*3
        draw_x, draw_y = int(x)+490, int(z)+490
        if cv2.waitKey(1) & 0xff == ord('q'):
            break

    #    n,e,d = angle[0], angle[1], angle[2]

      #  draw_x, draw_y = int(x)+490, int(y)+490
        cv2.circle(location, (draw_x,draw_y), 1, (0,0,255), 1)
        cv2.rectangle(location, (10, 20), (900, 60), (0,0,0), -1)    

        x0 = x/82.5
        y0 = y/82.5
        z0 = z/82.5
        text = "Coordinates: x=%2fm y=%2fm z=%2fm  "%(x0,y0,z0)
        print("x=%2fm y=%2fm z=%2fm " % (x0, y0,z0), file = f)
        if i%50 == 0:
            cv2.imwrite('map.jpg',location)
        cv2.putText(location, text, (20,40), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, 8)
        cv2.imshow('location', location)
        cv2.waitKey(1)
    i = i + 1
cv2.destroyWindow('MyCamera')
camcam.release()    

f.close()

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234


error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\calib3d\src\five-point.cpp:648: error: (-215:Assertion failed) E.cols == 3 && E.rows == 3 in function 'cv::decomposeEssentialMat'
